## **Deep Learning for Causal Inference by Vikas Ramachandra**

Applying the concepts from the "Deep Learning for Causal Inference" paper authored by Vikas Ramachandra to the data_for_churn_analysis dataset

link to the paper: <a>https://arxiv.org/abs/1803.00149</a>

In [1]:
# DA
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ML
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Metrics
from sklearn.metrics import accuracy_score, mean_absolute_percentage_error


In [2]:
# Loading dataset
df = pd.read_csv('data_for_churn_analysis.csv')

In [3]:
df.shape

(104143, 18)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104143 entries, 0 to 104142
Data columns (total 18 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   device                                 104025 non-null  object 
 1   first_payment_amount                   104143 non-null  int64  
 2   age                                    104001 non-null  float64
 3   city                                   98301 non-null   object 
 4   number_of_cards                        103671 non-null  float64
 5   payments_initiated                     103671 non-null  float64
 6   payments_failed                        103671 non-null  float64
 7   payments_completed                     103671 non-null  float64
 8   payments_completed_amount_first_7days  103671 non-null  float64
 9   reward_purchase_count_first_7days      80879 non-null   float64
 10  coins_redeemed_first_7days             103671 non-null  

In [5]:
# null values?
print(f"{df.isnull().sum().sum()=}")
print(f"perc of dataset missing {df.isnull().sum().sum()/df.shape[0]}")

df.isnull().sum().sum()=37490
perc of dataset missing 0.3599857887712088


In [6]:
# proportion of people who have churned
df['is_churned'].value_counts(normalize=True)

is_churned
0    0.713192
1    0.286808
Name: proportion, dtype: float64

### 1. **Impact of Referrals on Customer Acquisition and Retention**:
   - Research Question: Do customers acquired through referrals (`is_referral`) exhibit different behaviors and retention rates compared to non-referred customers?
   - Treatment: Customer acquisition through referrals
   - Outcome: Customer behavior (e.g., `payments_initiated`, `payments_completed`, `visits_feature_1`, `visits_feature_2`) and churn (`is_churned`)
   - Potential Confounders: `device`, `age`, `city`, `number_of_cards`, `payments_failed`, `payments_completed_amount_first_7days`, `reward_purchase_count_first_7days`, `coins_redeemed_first_7days`, `given_permission_1`, `given_permission_2`

$$Y_i = f(T_i, X_i, \epsilon_i)$$

Where:

- $Y_i$ represents the outcome variable `is_churned` for customer $i$
- $T_i$ is the treatment variable `is_referral`, indicating whether customer $i$ was acquired through a referral
- $X_i$ represents the vector of potential confounding variables for customer $i$, such as `device`, `age`, `city`, `number_of_cards`, `payments_failed`, `payments_completed_amount_first_7days`, `reward_purchase_count_first_7days`, `coins_redeemed_first_7days`, `given_permission_1`, `given_permission_2`
- $\epsilon_i$ is the error term, accounting for unobserved factors affecting the outcome
- $f$ is an unknown function that maps the treatment, confounders, and error term to the outcome

In [7]:
# -------------- DATA PREPROCESSING --------------------

# Encoding using label encoding technique
obj_cols = df.select_dtypes(include='object').columns # grabs object dtypes columns
le = LabelEncoder() # creates LabelEncoder instance
for col in obj_cols:
    df[col] = le.fit_transform(df[col]) # encodes each column

df['is_referral'] = np.where(df['is_referral'] == True, 1, 0)


# -------------- IMPUTING MISSING VALUES --------------------
missing_cols = df.columns[df.isna().any()].tolist()
for col in missing_cols:
    df[col] = df[col].fillna(df[col].mean())


# -------------- DATA SPLIT --------------------

confounders = [
    'device',
    'age',
    'city',
    'number_of_cards',
    'payments_failed',
    'payments_completed_amount_first_7days',
    'reward_purchase_count_first_7days',
    'coins_redeemed_first_7days',
    'given_permission_1',
    'given_permission_2',
    'is_referral' # treatment
]

y = df['is_churned']
X = df[confounders]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)
Y_train, Y_test = train_test_split(y, test_size=0.2, random_state=42)

### 1. **Generalized Neighbor Matching using Autoencoders**

The paper proposes using autoencoders, a type of deep neural network, for dimensionality reduction while preserving the local neighborhood structure of the data. This is useful for generalized neighbor matching to estimate individual treatment effects (ITEs).

The key points are:

* In high dimensions, traditional neighbor matching methods like k-nearest neighbors struggle
* Autoencoders can learn a low-dimensional representation that captures the manifold structure
* This low-dimensional encoding preserves local neighborhoods for accurate neighbor identification
* Experiments show autoencoders outperform methods like manifold learning for ITE estimation

#### **PCA**

#### **Manifold Learning**

#### **Autoencoder**

In [10]:
input_dim = X.shape[1]  # Features count

# Input layer
input_layer = Input(shape=(input_dim, ))

# Encoder: Reduce dimensionality
encoded = Dense(64, activation='relu')(input_layer)
encoded = Dense(32, activation='relu')(encoded)

# Decoder: Reconstruct the input
decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(input_dim, activation='sigmoid')(decoded)

# Autoencoder model
autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

In [11]:
autoencoder.fit(X_train, X_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(X_test, X_test))


Epoch 1/50
326/326 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 17951.0527 - val_loss: 18847.6953
Epoch 2/50
326/326 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 19158.2441 - val_loss: 18847.6562
Epoch 3/50
326/326 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 18545.9727 - val_loss: 18847.6270
Epoch 4/50
326/326 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 18116.4785 - val_loss: 18847.5996
Epoch 5/50
326/326 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 18266.0293 - val_loss: 18847.5957
Epoch 6/50
326/326 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 18893.5430 - val_loss: 18847.5723
Epoch 7/50
326/326 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 18223.2090 - val_loss: 18847.5684
Epoch 8/50
326/326 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 18491.7422 - val_loss: 18847.5645
Epoch 9/50
326/326 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 18212.2676 - val_loss: 18847.5645
Epoch 10/50
326/326 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 18069.1270 - val_loss: 18847.5996
Epoch 11/50
326/326 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss

In [44]:
encoder = Model(input_layer, encoded)
X_train_encoded = encoder.predict(X_train)
X_test_encoded = encoder.predict(X_test)
X_encoded = encoder.predict(X)

3255/3255 [==============================] - 5s 2ms/step


#### **Evaluating Each Method (PCA, Manifold and Autoencoder) based on the paper**


Next, to compare B. manifold learning and C. autoencoders, We also compute the estimated treatment effect for each point (ITE), and the average absolute error of ITE for B. manifold learning and C. Autoencoder, over all the data points in the test set.
- Mean Absolute error (ITE,autoencoder: 3.7127,
- Mean absolute error (ITE, Manifold learning): 4.4540
- Thus, autoencoder error is 20.27% lesser than manifold learning estimate for the ITE.

---

### 2. **Deep Neural Networks (DNNs) for Propensity Score Matching**

Propensity score matching is a popular technique, but traditionally uses logistic regression for propensity score estimation. The paper proposes using deep neural network classifiers instead, presenting a model called PropensityNet.

The key points are:

* DNNs can potentially capture complex non-linear relationships better than logistic regression
* PropensityNet is trained to estimate propensity scores as a binary classification problem
* Experiments show PropensityNet outperforms logistic regression for propensity score estimation
* This leads to better matching of treated and untreated units for ITE calculation

In [46]:
# Regular Logistic Regression
log_reg = LogisticRegression(max_iter = 10000).fit(X_train_encoded, Y_train)
preds = log_reg.predict_proba(X_encoded)

df_copy = df.copy()
df_copy.loc[:, 'propensity_score'] = preds[:, 1]

In [47]:
df_copy.groupby('is_churned')['propensity_score'].mean()

is_churned
0    0.237044
1    0.411856
Name: propensity_score, dtype: float64

In [55]:
# assuming a 0.5 threshold
accuracy_score(y, (preds[:, 1] > 0.5).astype('int'))

0.7676464092641848

In [56]:
1 - accuracy_score(y, (preds[:, 1] > 0.5).astype('int'))

0.23235359073581519

In [57]:
X.shape

(104143, 11)

---

In [58]:
# -------------- PROPENSITY NET --------------------

model = nn.Sequential(
                nn.Linear(in_features=11, out_features=64),
                nn.ReLU(),
                nn.Linear(in_features=64, out_features=64),
                nn.ReLU(),
                nn.Linear(in_features=64, out_features=64),
                nn.ReLU(),
                nn.Linear(in_features=64, out_features=32),
                nn.ReLU(),
                nn.Linear(in_features=32, out_features=16),
                nn.ReLU(),
                nn.Dropout(p=0.3),
                nn.Linear(in_features=16, out_features=1),
                nn.Sigmoid()
            )


# -------------- LOSS FUNCTION (Binaty Cross Entropy) & OPTIMIZER (Adam) --------------------
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


# ------- Tensors & Dataloaders --------------
# converting to pytorch tensors
X_tensor = torch.tensor(X.to_numpy(), dtype=torch.float32)
y_tensor = torch.tensor(y.to_numpy(), dtype=torch.float32)

# Dataloaders
train_dataset = TensorDataset(X_tensor, y_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=220, shuffle=True)


In [59]:
total_params = sum(p.numel() for p in model.parameters())
print("Total number of parameters:", total_params)


Total number of parameters: 11713


In [60]:
# -------------- TRAINING LOOP --------------------

EPOCHS = 20
for epoch in range(EPOCHS):
    epoch_loss = 0
    for inputs, label in train_dataloader:

        # ---- FORWARD PASS -----
        pred = model(inputs)
        pred = pred.squeeze(1)
        loss = criterion(pred, label)

        # ----- BACKPROPAGATION -----
        loss.backward() # calculating gradients
        optimizer.step() # multiplies learning_rate * gradient, which is your step size or by how much you update the weights

        # loss
        epoch_loss += loss.item()
    epoch_loss /= len(train_dataloader)
    print("epoch: {0} training loss {1}".format(epoch, epoch_loss))

epoch: 0 training loss 0.6417275298748338
epoch: 1 training loss 0.6035272799715211
epoch: 2 training loss 0.6261582033795143
epoch: 3 training loss 0.6186024529018482
epoch: 4 training loss 0.6038723844516126
epoch: 5 training loss 0.6258904630876292
epoch: 6 training loss 0.6078912219669246
epoch: 7 training loss 0.6118009460123279
epoch: 8 training loss 0.6243796401386019
epoch: 9 training loss 0.6033857540239261
epoch: 10 training loss 0.6184191579305673
epoch: 11 training loss 0.6130916148046904
epoch: 12 training loss 0.6068442858724151
epoch: 13 training loss 0.6232246962151949
epoch: 14 training loss 0.6037074091691005
epoch: 15 training loss 0.6162571190278742
epoch: 16 training loss 0.61226324491863
epoch: 17 training loss 0.607340791180164
epoch: 18 training loss 0.6207969548958766
epoch: 19 training loss 0.6026674624736802


In [61]:
# -------------- EVALUATING MODEL --------------------
# Make predictions
model.eval()  # Set the model to evaluation mode
predictions = []
with torch.no_grad():  # Disable gradient tracking during inference
    for batch in train_dataloader:
        inputs, labels = batch
        pred = model(inputs)
        pred = pred.squeeze(1)  # If your model outputs a single value per sample
        predictions.append(pred)


# Concatenate predictions into a single tensor
predictions_tensor = torch.cat(predictions)

# Convert predictions to numpy array
predictions_array = predictions_tensor.numpy()

In [62]:
predictions_array

array([0.34013438, 0.34013438, 0.34013438, ..., 0.34013438, 0.34013438,
       0.34013438], dtype=float32)

#### **Evaluating Each Method (LogisticRegression, PropensityNet) based on the paper**


- Evaluate them based on accuracy